In [1]:
root = "C:/Users/egorg/Projects/MemcoinsTradingBotPolygon"; import sys; sys.path.append(root)

In [2]:
import pandas as pd
import numpy as np
import src.utils as util

from src.env.data import ScalarFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor

In [3]:
data_path = f"{root}/data"
shift = int(180 / 60)
datasets = util.load_datasets(data_path, shift)
len(datasets)

5049

In [4]:
df = util.dataset_to_dataframe(datasets)
df = df[df["trial_rank"] == 0]
df

,dataset,std,rstd,ma_distance_ratio,rsi_crossovers,rsi_crossunders,entropy,regression_slope,regression_sqr,volatility_clustering,...,drawdown_convexity,drop_velocity,regime,trial_rank,profit,detection_period,drop_threshold,reversal_threshold,tp_pct,sl_pct
0,0_10,6.523627e-08,0.136456,0.048904,1,0,3.023363,0.002367,0.784891,0.045169,...,0.000027,0.711631,0,0,1005.423851,5,0.0460,0.03290,0.0491,0.00751
20,0_11,3.956624e-08,0.080538,0.027044,1,0,3.189496,-0.001863,0.944745,-0.086710,...,-0.000034,0.562611,1,0,1204.695376,5,0.1185,0.03895,0.0734,0.03944
40,0_12,2.014306e-08,0.044340,0.019405,1,0,2.179471,-0.000821,0.972760,-0.026485,...,0.000006,0.347130,2,0,1307.900000,3,0.0522,0.02336,0.0469,0.02719
60,0_13,2.400482e-08,0.056238,0.017800,1,0,2.219783,-0.001350,0.980654,-0.048117,...,-0.000014,0.271950,3,0,1365.978294,4,0.0509,0.02897,0.0327,0.02457
80,0_14,1.724124e-08,0.042506,0.024766,1,0,2.412590,-0.000219,0.966406,0.429391,...,-0.000012,2.363053,4,0,1144.787500,6,0.0912,0.07686,0.0964,0.08304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100880,9_4,2.109971e-07,0.085310,0.021940,4,0,2.649813,-0.002241,0.866363,0.056508,...,-0.000018,0.431340,84,0,1314.253231,6,0.0425,0.01345,0.0374,0.03294
100900,9_5,1.526912e-07,0.067886,0.025938,4,0,1.965845,-0.001244,0.564301,0.122511,...,0.000015,0.268158,425,0,1040.690137,2,0.0491,0.01573,0.0721,0.00931
100920,9_6,2.925304e-07,0.120796,0.043184,2,1,1.646462,0.002239,0.470154,-0.016312,...,-0.000017,0.775788,199,0,1142.515861,4,0.0357,0.01171,0.0803,0.01613
100940,9_7,1.469328e-07,0.054975,0.034891,0,1,1.963904,0.000559,0.368129,-0.018669,...,-0.000016,0.204268,92,0,1207.897028,4,0.0310,0.01410,0.0689,0.04989


In [12]:
features_selected: list[str] = ["path_length_ratio", "drop_velocity", "max_drawdown_ratio", "rstd", "regression_slope", "ma_distance_ratio"]
targets = ["detection_period", "drop_threshold", "reversal_threshold", "tp_pct", "sl_pct"]

In [6]:
features: list[list[np.float64 | int]] = []
for dataset in datasets:
    if dataset.features is None: continue
    scalar: ScalarFeatures = dataset.features.scalar
    buf: list[np.float64 | int] = []
    for e in ["detection_period", "drop_threshold", "reversal_threshold", "tp_pct", "sl_pct"]:
        try:
            buf.append(getattr(scalar, e))
        except AttributeError as _:
            buf.append(getattr(dataset.strategy_params[0], e))
    features.append(buf)
scaler = StandardScaler()
x = np.array(features, dtype=np.float64)
x_scaled = scaler.fit_transform(x)

In [7]:
k = 10
kmeans = KMeans(n_clusters=k, random_state=42)
labels = kmeans.fit_predict(x_scaled)
# df.insert(1, "regime", labels)
df["regime"] = labels

In [8]:
df = df[df["profit"] >= 1005]

In [9]:
df["drop_reversal_ratio"] = df["drop_threshold"] / df["reversal_threshold"]
df["tp_sl_ratio"] = df["tp_pct"] / df["sl_pct"]
df

C:\Users\egorg\AppData\Local\Temp\ipykernel_99352\825991508.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["drop_reversal_ratio"] = df["drop_threshold"] / df["reversal_threshold"]
C:\Users\egorg\AppData\Local\Temp\ipykernel_99352\825991508.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tp_sl_ratio"] = df["tp_pct"] / df["sl_pct"]


,dataset,std,rstd,ma_distance_ratio,rsi_crossovers,rsi_crossunders,entropy,regression_slope,regression_sqr,volatility_clustering,...,regime,trial_rank,profit,detection_period,drop_threshold,reversal_threshold,tp_pct,sl_pct,drop_reversal_ratio,tp_sl_ratio
0,0_10,6.523627e-08,0.136456,0.048904,1,0,3.023363,0.002367,0.784891,0.045169,...,7,0,1005.423851,5,0.0460,0.03290,0.0491,0.00751,1.398176,6.537949
20,0_11,3.956624e-08,0.080538,0.027044,1,0,3.189496,-0.001863,0.944745,-0.086710,...,1,0,1204.695376,5,0.1185,0.03895,0.0734,0.03944,3.042362,1.861055
40,0_12,2.014306e-08,0.044340,0.019405,1,0,2.179471,-0.000821,0.972760,-0.026485,...,6,0,1307.900000,3,0.0522,0.02336,0.0469,0.02719,2.234589,1.724899
60,0_13,2.400482e-08,0.056238,0.017800,1,0,2.219783,-0.001350,0.980654,-0.048117,...,7,0,1365.978294,4,0.0509,0.02897,0.0327,0.02457,1.756990,1.330891
80,0_14,1.724124e-08,0.042506,0.024766,1,0,2.412590,-0.000219,0.966406,0.429391,...,4,0,1144.787500,6,0.0912,0.07686,0.0964,0.08304,1.186573,1.160886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100880,9_4,2.109971e-07,0.085310,0.021940,4,0,2.649813,-0.002241,0.866363,0.056508,...,7,0,1314.253231,6,0.0425,0.01345,0.0374,0.03294,3.159851,1.135398
100900,9_5,1.526912e-07,0.067886,0.025938,4,0,1.965845,-0.001244,0.564301,0.122511,...,3,0,1040.690137,2,0.0491,0.01573,0.0721,0.00931,3.121424,7.744361
100920,9_6,2.925304e-07,0.120796,0.043184,2,1,1.646462,0.002239,0.470154,-0.016312,...,9,0,1142.515861,4,0.0357,0.01171,0.0803,0.01613,3.048676,4.978301
100940,9_7,1.469328e-07,0.054975,0.034891,0,1,1.963904,0.000559,0.368129,-0.018669,...,9,0,1207.897028,4,0.0310,0.01410,0.0689,0.04989,2.198582,1.381038


In [13]:
results = []
for i in range(200):
    df = df[df["trial_rank"] == 0].copy()
    dfr = df[df["regime"] == i]
    if len(dfr) < 10: continue
    x = dfr[features_selected].values.astype(np.float64)
    y = dfr[targets].values.astype(np.float64)
    mask = np.isfinite(x).all(axis=1) & np.isfinite(y).all(axis=1)
    x, y = x[mask], y[mask]

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    base_model = LGBMRegressor(
        n_estimators=512,
        learning_rate=0.001,
        max_depth=8,
        num_leaves=8,
        subsample=0.9,
        colsample_bytree=0.9,
        random_state=42
    )
    model = MultiOutputRegressor(base_model)
    model.fit(x_train, y_train)

    row = {}
    y_pred = model.predict(x_test)
    for i, name in enumerate(targets):
        mae = mean_absolute_error(y_test[:, i], y_pred[:, i])
        row[name] = mae
    results.append(row)
results = pd.DataFrame(results)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029781 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 613
[LightGBM] [Info] Number of data points in the train set: 306, number of used features: 6
[LightGBM] [Info] Start training from score 2.130719
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000236 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 613
[LightGBM] [Info] Number of data points in the train set: 306, number of used features: 6
[LightGBM] [Info] Start training from score 0.045166
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 613
[LightGBM] [Info] Number of data points in the train set: 306, number of used features: 6
[LightGBM] [Info] Start training from s

c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserW

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000175 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 451
[LightGBM] [Info] Number of data points in the train set: 223, number of used features: 6
[LightGBM] [Info] Start training from score 4.502242
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000375 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 451
[LightGBM] [Info] Number of data points in the train set: 223, number of used features: 6
[LightGBM] [Info] Start training from score 0.082343
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000167 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 451
[LightGBM] [Info] Number of data points in the train set: 223, nu

c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserW

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 167
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 6
[LightGBM] [Info] Start training from score 5.125000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

...

c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserW

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 113
[LightGBM] [Info] Number of data points in the train set: 54, number of used features: 6
[LightGBM] [Info] Start training from score 2.148148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserW

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000295 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1127
[LightGBM] [Info] Number of data points in the train set: 564, number of used features: 6
[LightGBM] [Info] Start training from score 0.041072
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000208 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1127
[LightGBM] [Info] Number of data points in the train set: 564, number of used features: 6
[LightGBM] [Info] Start training from score 0.017930
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000278 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1127
[LightGBM] [Info] Number of data points in the train set: 564, number of used features: 6
[LightGBM] [Info] Start training fro

c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\egorg\Projects\MemcoinsTradingBotPolygon\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserW

In [14]:
results

,detection_period,drop_threshold,reversal_threshold,tp_pct,sl_pct
0,0.716811,0.011724,0.008511,0.008302,0.009757
1,0.881166,0.011905,0.010400,0.012578,0.011384
2,0.799714,0.015383,0.015498,0.016147,0.009543
3,0.663681,0.010377,0.007955,0.012737,0.010139
4,1.226749,0.014093,0.016708,0.008961,0.013815
5,0.707883,0.013915,0.008695,0.007847,0.010480
6,0.696820,0.011880,0.009659,0.010992,0.008148
7,0.735751,0.011931,0.008608,0.009778,0.007104
8,0.729468,0.018420,0.014465,0.015878,0.010877
9,0.675960,0.008058,0.006321,0.012201,0.011262
